In [ ]:
import numpy as np
import pydicom
import os
import glob
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops
from sklearn import mixture
from matplotlib import gridspec 
import pandas as pd
from skimage.morphology import erosion, dilation
from skimage.morphology import disk
from skimage.morphology import ball
from skimage.morphology import reconstruction
from skimage.feature import peak_local_max
from time import time
from skimage.filters import laplace
from skimage.morphology import watershed, label
from skimage.filters import threshold_otsu
from skimage import exposure
import cv2
import matplotlib.image as mpimg
from PIL import Image 
from skimage.segmentation import flood_fill
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import xlsxwriter

In [ ]:
NAMES = [215379,218492,218493,218494,218508,218510,218527,218529,218542,218552,218562,218563,315402,315404,315405,315427,315439]  

In [ ]:
def distance(a,b):
    return np.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

In [ ]:
# Detectar carina

CARINES = [] # (AP,LR,SI) 
for ii in range(len(NAMES)):
    # Llegir carpeta amb les imatges de TAC      
    data_path = 'C:\\Users\\InmaV\\Downloads\\P' + str(NAMES[ii])
    g = glob.glob(data_path+'\\' + '*.dcm')

    l = [] # Afegir els valors de píxel de cada imatge a la llista l
    for n in g:
        ds = pydicom.dcmread(n)
        image2D = ds.pixel_array
        l.append(image2D.astype(np.int16))

    sx = ds.PixelSpacing[0]; sy = ds.PixelSpacing[1]; sz = ds.SpacingBetweenSlices
    spacing = [sx,sy,sz] # Espaiat en cada direcció
    L = np.copy(l); L[L < 0] = 0 # Treure les intensitats negatives, canviar-les per 0 
    size_vol = L.shape # Dimensions de L

    Ml2 = L > 500 # Llindar global a 500
    Ml2 = 1 - Ml2 # Inversió de la màscara

    IMAGE = []
    for j in range(size_vol[0]):
        selem = disk(2.9)
        eroded = erosion(Ml2[j], selem) # Erosió amb disc de radi 2.9 com a element estructurant

        label_image, nregions = label(eroded,return_num=True) # Etiquetar les diferents regions
        props = regionprops(label_image) # Obtenir les propietats de les diferents regions

        area = np.zeros(nregions)
        for i in range(nregions):
            area[i] = props[i].area
        ind_small = np.where(area < 10)

        for i in ind_small[0]: # Eliminar regions menors a 10 píxels
            iregion = i + 1
            label_image[label_image == iregion] = 0

        label_image2, nregions = label(label_image,return_num=True)
        props = regionprops(label_image2)

        area = np.zeros(nregions)
        for i in range(nregions): # Eliminar regions majors a 700 píxels
            area[i] = props[i].area
        ind_large = np.where(area > 700)

        for i in ind_large[0]:
            iregion = i + 1
            label_image2[label_image2 == iregion] = 0

        selem = disk(4)
        label_image2 = dilation(label_image2, selem)
        label_image3, nregions = label(label_image2,return_num=True)

        IMAGE.append(label_image3) # Crear una nova llista amb totes les imatges processades

    # Inici tràquea
    s = round(size_vol[0]*0.3); s_ini = s # Imatge corresponent al 30% del total
    label_image, nregions = label(IMAGE[s],return_num=True) # Etiquetar la imatge
    props = regionprops(label_image)

    CI = ((round(size_vol[1]/2),round(size_vol[2]/2))) # Centre de la imatge
    area = np.zeros(nregions); D = np.zeros(nregions); v_i = []; v_dist = []
    for i in range(nregions):
        area[i]=props[i].area
        cm = props[i].centroid
        D[i] = distance(cm,CI)
        # Condicions per la tràquea: àrea entre 100 i 850 píxels i a menys de 50 píxels de distància del centre
        if (props[i].area > 100) & (props[i].area < 850) & (D[i] < 50):
            v_i.append(i)
            v_dist.append(D[i])

    minim = np.where(v_dist == np.amin(v_dist))[0][0]; index = v_i[minim] # si hi ha més d'una àrea a prop del centre, agafa la més propera
    area_trachea = props[index].area; area_ini = area_trachea; ecc_trachea = props[index].eccentricity
    CM_trachea = props[index].centroid  
    print('First trachea is in image ' + str(s) + ' with center at (ap, lr) = (' + str(int(CM_trachea[0])) + ', ' + str(int(CM_trachea[1])) + ')')

    index = np.where(area != area_trachea) # Mostrar només la tràquea a la imatge 
    for i in index[0]:
        iregion = i + 1
        label_image[label_image == iregion] = 0
    
    # Imatge amb el TAC original i l'estructura corresponent a la tràquea
    fig = plt.figure(figsize=(10, 7))
    gs = gridspec.GridSpec(1, 2)

    ax1 = fig.add_subplot(gs[0,0])
    ax1.imshow(L[s], cmap=plt.cm.gray)
    ax1.set_axis_off()
    ax1.set_title('Image Total/4')

    ax2 = fig.add_subplot(gs[0,1])
    ax2.imshow(label_image, cmap=plt.cm.gray)
    ax2.set_axis_off()
    ax2.set_title('Trachea')
    
    # Tobar carina: quan l'àrea de la nova tràquea és inferior al 70% o superior al 250% de l'anterior
    IMAGE2=[]
    while ((area_trachea/area_ini > 0.7) and (area_trachea/area_ini < 2.5)):
        s = s + 1; area_ini = area_trachea; CM_ini = np.copy(CM_trachea)
        label_image, nregions = label(IMAGE[s],return_num=True) # Etiquetar imatge
        props = regionprops(label_image)

        area = np.zeros(nregions); D = np.zeros(nregions); CM = np.zeros((2,nregions))
        for i in range(nregions):
            area[i] = props[i].area
            cm = props[i].centroid; CM[0,i] = cm[0]; CM[1,i] = cm[1]
            D[i] = distance(cm,CM_trachea) # Distància de cada regió trobada a la tràquea anterior

        D2 = np.sort(D); DD = D2[0] # Ordenar distàncies de menor a major i escollir com a nova tràquea la primera (més propera a l'anterior)
        index = np.where(D == DD); area_trachea = area[index[0]]
        CM_trachea=np.zeros((2,1)); CM_trachea[0] = CM[0,index[0]][0]; CM_trachea[1] = CM[1,index[0]][0]
        index2 = np.where(D != DD)
        
        for i in index2[0]:
            iregion = i+1
            label_image[label_image==iregion]=0
        IMAGE2.append(label_image)
    
    # Imatge amb el TAC original i l'estructura corresponent a la carina 
    fig = plt.figure(figsize=(10, 7))
    gs = gridspec.GridSpec(1, 2)

    ax1 = fig.add_subplot(gs[0,0])
    ax1.imshow(L[s-1], cmap=plt.cm.gray)
    ax1.set_axis_off()
    ax1.set_title('Input Image')

    img = np.copy(IMAGE2[-2]); img[img > 0] = 1
    ax2 = fig.add_subplot(gs[0,1])
    ax2.imshow(img, cmap=plt.cm.gray)
    ax2.set_axis_off()
    ax2.set_title('Carina')
    print('Carina is in image ' + str(s-1) + ' with center at (ap, lr) = (' + str(int(CM_ini[0][0])) + ', ' + str(int(CM_ini[1][0])) + ')')
    CARINES.append([int(CM_ini[0][0]), int(CM_ini[1][0]), int(s - 1)])


In [ ]:
NAMES = [215379,218492,218493,218494,218508,218510,218529,218542,218552,218562,218563,315402,315404,315439]  
CARINES = [[263, 269, 115], [224, 256, 128], [242, 247, 151], [259, 246, 109], [263, 253, 107],
 [229, 260, 115], [254, 249, 129], [230, 283, 96], [238, 270, 128], [238, 253, 131],
 [261, 248, 122], [266, 273, 123], [262, 257, 122], [233, 245, 119]]

In [ ]:
# Ajustar la posició de la carina al ground truth (Excel radiòleg)

threshold = 1100 # llindar
MATRIU_CARINES = []; MATRIU_TP  =[]; MATRIU_FP = []; MATRIU_FN = []
for jj in range(len(NAMES)):
    print('Pulmó P' + str(NAMES[jj]))
    # M_carina: llista amb les diferents posicions que es provaran com a carina
    M_carina= []; name = NAMES[jj]
    carina_detectada = np.array(CARINES[jj])
    v1 = np.linspace(carina_detectada[0] - 1, carina_detectada[0] + 13, 15) # AP: buscar 1 píxel amunt i 13 avall 
    v2 = np.linspace(carina_detectada[1] - 3, carina_detectada[1] + 32, 36) # LR: buscar 3 píxels a la dreta i 32 a l'esquerra 
    v3 = np.linspace(carina_detectada[2], carina_detectada[2] + 6, 7) # SI: buscar 6 TACs endavant 
    for i in range(len(v1)):
        for j in range(len(v2)):
            for k in range(len(v3)):
                c = (v1[i],v2[j],v3[k])
                M_carina.append(c)
    
    # Llegir carpeta amb les imatges de TAC      
    data_path = 'C:\\Users\\InmaV\\Downloads\\P' + str(NAMES[jj])
    g = glob.glob(data_path+'\\' + '*.dcm')

    l = [] # Afegir els valors de píxel de cada imatge a la llista l
    for n in g:
        ds = pydicom.dcmread(n)
        image2D = ds.pixel_array
        l.append(image2D.astype(np.int16))

    sx = ds.PixelSpacing[0]; sy = ds.PixelSpacing[1]; sz = ds.SpacingBetweenSlices
    spacing = [sx,sy,sz] # Espaiat en cada direcció
    L = np.copy(l); L[L < 0] = 0 # Treure les intensitats negatives, canviar-les per 0 
    size_vol = L.shape # Dimensions de L

    # Detectar volum pulmonar       
    I0 = L[0] # Primera imatge del TAC
    Ib = (I0 >= 600) # Llindar global a 600

    SUPORT, nregions = label(Ib,return_num=True) # Etiquetar les diferents regions
    props = regionprops(SUPORT)

    area = np.zeros(nregions)
    for i in range(nregions):
        area[i]=props[i].area
    area_s = np.sort(area); AREA = area_s[-1] # El suport és l'àrea més gran
    index = np.where(area!=AREA)

    for i in index[0]:
        iregion=i+1
        SUPORT[SUPORT==iregion]=0
    SUPORT[SUPORT > 0] = 1 # Imatge només amb l'estructura corresponent al suport de la primera imatge


    If = L[-1] # Última imatge del TAC
    Ib = (If >= 600)&(If <= 850) # 2 llindars globals: inferior, a 600, i superior, a 850

    SUPORT2, nregions = label(Ib,return_num=True)
    selem = disk(2)
    SUPORT2 = dilation(SUPORT2, selem)
    SUPORT2, nregions = label(SUPORT2,return_num=True)
    props = regionprops(SUPORT2)

    area = np.zeros(nregions)
    for i in range(nregions):
        area[i]=props[i].area
    area_s = np.sort(area); AREA = area_s[-1] # El suport és l'àrea més gran
    index = np.where(area!=AREA)

    for i in index[0]:
        iregion=i+1
        SUPORT2[SUPORT2==iregion]=0
    SUPORT2[SUPORT2 > 0] = 1 # Imatge només amb l'estructura corresponent al suport de l'última imatge
    MASK = SUPORT + SUPORT2 # Màscara: suma dels 2 suports trobats
    

    thresh = threshold_otsu(L)
    binary = L > thresh

    CI = ((round(size_vol[1]/2),round(size_vol[2]/2))); I2=[]
    for i in range(size_vol[0]):
        I = binary[i] - MASK # Restar la màscara a cada imatge
        I[I > 0] = 1; I[I < 0] = 0 # 

        selem = disk(2)
        filled = erosion(I, selem) # Erosió, disc radi 2

        I2.append(filled)


    MI2 = np.asarray(I2, dtype=np.float32) # Convertir llista en matriu 3D per poder etiquetar
    MI2_label, nregions = label(MI2,return_num=True)
    selem = ball(2)
    MI2_label = dilation(MI2_label, selem) # Dilatació, esfera radi 2

    # Quins elements diferents hi ha a MI2_label i en quina quantitat
    unique_elements, counts_elements = np.unique(MI2_label, return_counts=True)
    counts_elements2 = np.sort(counts_elements)
    c = counts_elements2[-2] # Escollir el segon grup més abundant (el primer correspon al fons)
    index = np.where(counts_elements == c)

    MI2_label[MI2_label != index[0]] = 0 # Tot píxel que no pertany al volum pulmonar es transforma en 0
    MI2_label[MI2_label == index[0]] = 1 # Tot píxel que pertany al volum pulmonar es transforma en 1

    LL = L*MI2_label # Matriu només amb volum pulmonar

    # Trobar màxims locals, llindar absolut a 1000 i distància mínima de 2 píxels
    coordinates = peak_local_max(LL,indices=False,threshold_abs=threshold,min_distance=2)
    markers,nregions = label(coordinates,return_num=True) # Etiquetar regions

    props = regionprops(markers,L)  
    CM = np.zeros((nregions,3)) # Centroide ponderat de cada regió
    for i in range(nregions):
        cm = props[i].weighted_centroid 
        CM[i,0] = cm[0]; CM[i,1] = cm[1]; CM[i,2] = cm[2]

        
    # Llegir Excel radiòleg
    df = pd.read_excel('Experimental_Data.xlsx', sheet_name='Nódulos')
    numero = df["número"].to_numpy()[:945]; nom = df["cerdo"].to_numpy()[:945]; vlr = df["eje R/L"].to_numpy()[:945]; lr = df["Cifra R/L"].to_numpy()[:945] 
    vap = df["Eje A/P"].to_numpy()[:945]; ap = df["Cifra A/P"].to_numpy()[:945]; si = df["Posición mesa (I)"].to_numpy()[:945]; d = df["Diametro (mm)"].to_numpy()[:945]
    
    dfc = pd.read_excel('Experimental_Data.xlsx', sheet_name='Cerdos')
    nom_c = dfc["CERDO"].to_numpy()[:24]; vclr = dfc["Carina R/L"].to_numpy()[:24]; clr = dfc["Cifra R/L"].to_numpy()[:24] 
    vcap = dfc["Carina A/P"].to_numpy()[:24]; cap = dfc["Cifra A/P"].to_numpy()[:24]; csi = dfc["Cifra I"].to_numpy()[:24]

    NOM = NAMES[JJ]
    v_numero = []
    for i in range(len(nom)):
        if nom[i] == NOM:
            v_numero.append(numero[i])

    if np.shape(v_numero)[0] == 0: # si el pulmó no té lesions, totes les llistes són buides
        lrc = []
        apc = []
        zc = []
        D = []

    else:
        # Canviar signes coordenades lesions si cal
        LR = []; AP = []; SI = []; D = []
        for i in v_numero:
            i = int(i)
            if vlr[i-1] == 'R':
                lr[i-1] = -lr[i-1]
            if vap[i-1] == 'A':
                ap[i-1] = -ap[i-1]
            LR.append(lr[i-1]); AP.append(ap[i-1]); SI.append(si[i-1]); D.append(d[i-1])

        # Canviar signes coordenades carina si cal
        for i in range(len(nom_c)):
            if nom_c[i] == NOM:
                if vclr[i] == 'R':
                    clr[i] = -clr[i]
                if vcap[i] == 'A':
                    cap[i] = -cap[i]
                carina = (cap[i], clr[i], csi[i])

        # Ordenar tot en funció de coordenada SI
        c_tuples = list(zip(LR, AP, SI, D)) #llista amb coordenades (lr,ap,si) i diàmetre de cada lesió
        C = sorted(c_tuples, key=lambda c: c[2], reverse = True) # ordenar tot respecte SI

        # Coordenades radiòleg en mm respecte la carina
        LR, AP, SI, D = zip(*C) # separar cada coordenada en un vector

        # Coordenades radiòleg en mm respecte la carina (restar carina radiòleg, respecte a punt random)
        lrc = np.asarray(LR) - carina[1]
        apc = np.asarray(AP) - carina[0]
        sic = np.asarray(SI) - carina[2] 


    v_TP = []; v_FP = []; v_FN = []; v_mean_dist = []
    for ii in range(len(M_carina)): # Provar cada posició de la llista M_carina
        carina_p = M_carina[ii]
        CMap = []; CMlr = []; CMsi = []; CMap2 = []; CMlr2 = []; CMsi2 = []
        for i in range(len(CM)):
            CMap.append(CM[i,1]); CMlr.append(CM[i,2]); CMsi.append(CM[i,0])
            # Coordenades respecte la carina i en mm
            CMap2.append((CM[i,1] - carina_p[0])*spacing[0]) 
            CMlr2.append((CM[i,2] - carina_p[1])*spacing[1])
            CMsi2.append((CM[i,0] - carina_p[2])*spacing[2])


        c_tuples = list(zip(CMap, CMlr, CMsi, CMap2, CMlr2, CMsi2)) 
        C = sorted(c_tuples, key=lambda c: c[2]) # ordenar tot respecte SI 

        CMap, CMlr, CMsi, CMap2, CMlr2, CMsi2 = zip(*C) # separar cada coordenada en un vector

        lrpl = CMlr; appl = CMap; sipl = CMsi # coordenades en píxels
        lrl = CMlr2; apl = CMap2; sil = CMsi2 # coordenades en mm

        # Comparació lesions detectades amb les del radiòleg
        if np.shape(lrc)[0] == 0: 
            print('Quantitat de FP: ' + str(len(lrl))) 
            
        else:
            vexp = []; vjj = []; vSS = []; vii = []
            for i in range(len(lrpl)):
                vj = []; vS = []
                for j in range(len(lrc)):
                    # Distància de cada regió detectada a les lesions del radiòleg
                    S = np.sqrt((lrl[i] - lrc[j])**2 + (apl[i] - apc[j])**2 + (sil[i] - sic[j])**2)
                    vS.append(S)
                    vj.append(j)
                S_min = np.amin(vS); vSS.append(S_min) # Es relaciona una regió amb la lesió del radiòleg més propera
                index = np.where(vS == S_min)[0][0]; vjj.append(vj[index])
                vii.append(i)

            dist_min = 4 # mm
            c_tuples = list(zip(vjj, vSS, vii)) 
            C = sorted(c_tuples, key=lambda c: c[0]) # ordenar tot respecte vjj 
            vjj, vSS, vii = zip(*C) # separar cada coordenada en un vector
            vSS = np.asarray(vSS)

            # True Positives: lesions detectades a una distància menor a dist_min d'una del radiòleg
            index1 = np.where(vSS <= dist_min)
            vl_TP = []; ve_TP = []; v_dist = []
            for i in index1[0]:
                vl_TP.append(vii[i])
                ve_TP.append(vjj[i])
                v_dist.append(vSS[i])

            vectori = [] # Hi ha lesions del radiòleg assignades a 2 meves, em quedo amb la més propera
            for i in range(len(ve_TP)-1):
                if ve_TP[i] == ve_TP[i+1]:
                    S1 = vSS[i]
                    S2 = vSS[i+1]
                    if S1 < S2:
                        vectori.append(i)
                    elif S1 > S2:
                        vectori.append(i+1)

            count = 0; vl_FP = [] # primeres False Positives: aquestes "TP" descartades
            for i in vectori:
                ve_TP.remove(ve_TP[i-count])
                vl_FP.append(vl_TP[i-count])
                vl_TP.remove(vl_TP[i-count])
                v_dist.remove(v_dist[i-count])
                count = count + 1 # cada cop que es fa remove s'acurta el vector
            v_mean_dist.append(np.mean(v_dist)) # Llista amb distància mitjana entre les lesions detectades i les del radiòleg per cada carina

            # False Positives: lesions detectades a una distància major a dist_min d'una del radiòleg
            index22 = np.where(vSS >= dist_min)
            for i in index22[0]:
                vl_FP.append(vii[i])
            vl_FP = np.sort(vl_FP)

            # False Negatives: lesions detectades pel radiòleg però no per mi
            ve_FN = np.setxor1d(np.arange(0,len(sic)), ve_TP) # diferència entre 2 conjunts, elements que hi ha a (vector amb nº lesions) però no a ve_TP

            v_TP.append(len(vl_TP)); v_FP.append(len(vl_FP)); v_FN.append(len(ve_FN)) # llistes amb nombre total de TP, FP i FN

    # Millors carines
    M_carina2 = []; v_mean_dist2 = []; v_TP2 = []; v_FP2 = []; v_FN2 = []
    A = np.where(v_TP == np.amax(v_TP)) # Carines que proporcionen més TP
    for i in A[0]:
        M_carina2.append(M_carina[i])
        v_mean_dist2.append(v_mean_dist[i])
        v_TP2.append(v_TP[i]); v_FP2.append(v_FP[i]); v_FN2.append(v_FN[i])
    
    minim = np.amin(v_mean_dist2)
    B = np.where(v_mean_dist2 == minim) # S'escull carina que dóna menys distància lesions detectades - lesions radiòleg
    
    while minim > 2: # Volem que la distància mitjana sigui menor a 2.5 mm
        BB = np.where(v_mean_dist == minim)
        M_carina = np.array(M_carina)
        M_carina = np.delete(M_carina,BB[0][0],0) # Eliminar carina anterior
        v_mean_dist = np.delete(v_mean_dist,BB[0][0])
        v_TP = np.delete(v_TP,BB[0][0]); v_FP = np.delete(v_FP,BB[0][0]); v_FN = np.delete(v_FN,BB[0][0])

        M_carina2 = []; v_mean_dist2 = []; v_TP2 = []; v_FP2 = []; v_FN2 = []
        A = np.where(v_TP == np.amax(v_TP)) # Carines que donen més TP
        for i in A[0]:
            M_carina2.append(M_carina[i])
            v_mean_dist2.append(v_mean_dist[i])
            v_TP2.append(v_TP[i]); v_FP2.append(v_FP[i]); v_FN2.append(v_FN[i])

        minim = np.amin(v_mean_dist2)
        B = np.where(v_mean_dist2 == minim) # Carina amb menor distància mitjana

    
    for i in B[0]:
        print(v_TP2[i],M_carina2[i],v_mean_dist2[i])
        MATRIU_CARINES.append(M_carina2[i])
        MATRIU_TP.append(v_TP2[i])
        MATRIU_FP.append(v_FP2[i])
        MATRIU_FN.append(v_FN2[i])


In [ ]:
MCARINES = [(266.0, 271.0, 115.0),(228.0, 256.0, 131.0),(252.0, 248.0, 153.0),(265.0, 246.0, 113.0),
(269.0, 256.0, 113.0),(231.0, 258.0, 118.0),(261.0, 251.0, 132.0),(229.0, 251.0, 102.0),(243.0, 271.0, 132.0),
(240.0, 250.0, 134.0),(265.0, 246.0, 124.0),(278.0, 269.0, 125.0),(265.0, 254.0, 124.0),(246.0, 248.0, 125.0)]

In [ ]:
# Detectar lesions 

# Definir llistes
TOTAL_lrl = []; TOTAL_apl = []; TOTAL_sil = []; TOTAL_v_min_maxLR = []; TOTAL_v_min_maxAP = []; TOTAL_v_min_maxSI = []
TOTAL_dlr = []; TOTAL_dap = []; TOTAL_dsi = []; TOTAL_Ic = []; TOTAL_EDmm = []; TOTAL_YESNO = []; TOTAL_nom = []
TOTAL_lrpl = []; TOTAL_appl = []; TOTAL_sipl = []; TOTAL_AREA = []; TOTAL_EXT = []; TOTAL_STD = []; TOTAL_STD_norm = []
TOTAL_Ic_norm = []; TOTAL_DIF_I = []; TOTAL_ENTROPY = []; TOTAL_Imax = []; TOTAL_DIF_I2 = []; TOTAL_MAL = []
threshold = 1100 # llindar

# Carines que donen els millors resultats: 
NAMES = [215379,218492,218493,218494,218508,218510,218529,218542,218552,218562,218563,315402,315404,315439]  
CARINES = [(266.0, 271.0, 115.0),(228.0, 256.0, 131.0),(252.0, 248.0, 153.0),(265.0, 246.0, 113.0),
(269.0, 256.0, 113.0),(231.0, 258.0, 118.0),(261.0, 251.0, 132.0),(229.0, 251.0, 102.0),(243.0, 271.0, 132.0),
(240.0, 250.0, 134.0),(265.0, 246.0, 124.0),(278.0, 269.0, 125.0),(265.0, 254.0, 124.0),(246.0, 248.0, 125.0)]

threshold = 1100; v_TP = []; v_FP = []; v_FN = []

for ii in range(len(NAMES)):
    print(NAMES[ii])
    # Llegir carpeta amb les imatges de TAC
    data_path = 'C:\\Users\\InmaV\\Downloads\\P' + str(NAMES[JJ])
    g = glob.glob(data_path + '\\' + '*.dcm')

    l = [] # Afegir els valors de píxel de cada imatge a la llista l
    for n in g:
        ds = pydicom.dcmread(n)
        image2D = ds.pixel_array
        l.append(image2D.astype(np.int16))

    slr = ds.PixelSpacing[0]; sap = ds.PixelSpacing[1]; ssi = ds.SpacingBetweenSlices
    spacing = [slr,sap,ssi]
    L = np.copy(l); L[L < 0] = 0 # Treure les intensitats negatives, canviar-les per 0 
    size_vol = L.shape # dimensions de L

    # Detectar pulmons       
    I0 = L[0] # Primera imatge del TAC
    Ib = (I0 >= 600) # Llindar global a 600

    SUPORT, nregions = label(Ib,return_num=True) # Etiquetar les diferents regions
    props = regionprops(SUPORT)

    area = np.zeros(nregions)
    for i in range(nregions): 
        area[i]=props[i].area
    area_s = np.sort(area); AREA = area_s[-1] # El suport és l'àrea més gran
    index = np.where(area!=AREA)

    for i in index[0]:
        iregion=i+1
        SUPORT[SUPORT==iregion]=0
    SUPORT[SUPORT > 0] = 1 # Imatge només amb l'estructura corresponent al suport de la primera imatge


    If = L[-1] # Última imatge del TAC
    Ib = (If >= 600)&(If <= 850) # 2 llindars globals: inferior, a 600, i superior, a 850

    SUPORT2, nregions = label(Ib,return_num=True)
    selem = disk(2)
    SUPORT2 = dilation(SUPORT2, selem)
    SUPORT2, nregions = label(SUPORT2,return_num=True)
    props = regionprops(SUPORT2)

    area = np.zeros(nregions)
    for i in range(nregions):
        area[i]=props[i].area
    area_s = np.sort(area); AREA = area_s[-1] # El suport és l'àrea més gran
    index = np.where(area!=AREA)

    for i in index[0]:
        iregion=i+1
        SUPORT2[SUPORT2==iregion]=0
    SUPORT2[SUPORT2 > 0] = 1 # Imatge només amb l'estructura corresponent al suport de l'última imatge
    MASK = SUPORT + SUPORT2 # Màscara: suma dels 2 suports trobats
    

    thresh = threshold_otsu(L) # Llindar Otsu
    binary = L > thresh

    CI = ((round(size_vol[1]/2),round(size_vol[2]/2))); I2=[]
    for i in range(size_vol[0]):
        I = binary[i] - MASK # Restar la màscara a cada imatge
        I[I > 0] = 1; I[I < 0] = 0 # 

        selem = disk(2)
        filled = erosion(I, selem) # Erosió, disc radi 2

        I2.append(filled)


    MI2 = np.asarray(I2, dtype=np.float32) # Convertir llista en matriu 3D per poder etiquetar
    MI2_label, nregions = label(MI2,return_num=True)
    selem = ball(2)
    MI2_label = dilation(MI2_label, selem) # Dilatació, esfera radi 2

    # Quins elements diferents hi ha a MI2_label i en quina quantitat
    unique_elements, counts_elements = np.unique(MI2_label, return_counts=True)
    counts_elements2 = np.sort(counts_elements)
    c = counts_elements2[-2] # Escollir el segon grup més abundant (el primer correspon al fons)
    index = np.where(counts_elements == c)

    MI2_label[MI2_label != index[0]] = 0 # Tot píxel que no pertany al volum pulmonar es transforma en 0
    MI2_label[MI2_label == index[0]] = 1 # Tot píxel que pertany al volum pulmonar es transforma en 1

    LL = L*MI2_label # Matriu només amb volum pulmonar

    # Trobar lesions: màxims locals, llindar absolut a 1000 i distància mínima de 2 píxels
    coordinates = peak_local_max(LL,indices=False,threshold_abs=threshold,min_distance=2)
    markers,nregions = label(coordinates,return_num=True) # Etiquetar regions

    props = regionprops(markers,L)  
    CM = np.zeros((nregions,3)) # Centroide ponderat de cada regió
    for i in range(nregions):
        cm = props[i].weighted_centroid 
        CM[i,0] = cm[0]; CM[i,1] = cm[1]; CM[i,2] = cm[2]



    # Llegir Excel radiòleg
    df = pd.read_excel('Experimental_Data.xlsx', sheet_name='Nódulos')
    numero = df["número"].to_numpy()[:945]; nom = df["cerdo"].to_numpy()[:945]; vlr = df["eje R/L"].to_numpy()[:945]; lr = df["Cifra R/L"].to_numpy()[:945] 
    vap = df["Eje A/P"].to_numpy()[:945]; ap = df["Cifra A/P"].to_numpy()[:945]; si = df["Posición mesa (I)"].to_numpy()[:945]; d = df["Diametro (mm)"].to_numpy()[:945]
    
    dfc = pd.read_excel('Experimental_Data.xlsx', sheet_name='Cerdos')
    nom_c = dfc["CERDO"].to_numpy()[:24]; vclr = dfc["Carina R/L"].to_numpy()[:24]; clr = dfc["Cifra R/L"].to_numpy()[:24] 
    vcap = dfc["Carina A/P"].to_numpy()[:24]; cap = dfc["Cifra A/P"].to_numpy()[:24]; csi = dfc["Cifra I"].to_numpy()[:24]

    NOM = NAMES[JJ]
    v_numero = []
    for i in range(len(nom)):
        if nom[i] == NOM:
            v_numero.append(numero[i])

    if np.shape(v_numero)[0] == 0: # si el pulmó no té lesions, totes les llistes són buides
        lrc = []
        apc = []
        zc = []
        D = []

    else:
        # Canviar signes coordenades lesions si cal
        LR = []; AP = []; SI = []; D = []
        for i in v_numero:
            i = int(i)
            if vlr[i-1] == 'R':
                lr[i-1] = -lr[i-1]
            if vap[i-1] == 'A':
                ap[i-1] = -ap[i-1]
            LR.append(lr[i-1]); AP.append(ap[i-1]); SI.append(si[i-1]); D.append(d[i-1])

        # Canviar signes coordenades carina si cal
        for i in range(len(nom_c)):
            if nom_c[i] == NOM:
                if vclr[i] == 'R':
                    clr[i] = -clr[i]
                if vcap[i] == 'A':
                    cap[i] = -cap[i]
                carina = (cap[i], clr[i], csi[i])

        # Ordenar tot en funció de coordenada SI
        c_tuples = list(zip(LR, AP, SI, D)) #llista amb coordenades (lr,ap,si) i diàmetre de cada lesió
        C = sorted(c_tuples, key=lambda c: c[2], reverse = True) # ordenar tot respecte SI

        # Coordenades radiòleg en mm respecte la carina
        LR, AP, SI, D = zip(*C) # separar cada coordenada en un vector

        # Coordenades radiòleg en mm respecte la carina (restar carina radiòleg, respecte a punt random)
        lrc = np.asarray(LR) - carina[1]
        apc = np.asarray(AP) - carina[0]
        sic = np.asarray(SI) - carina[2] 


    carina_p = CARINES[ii]
    CMap = []; CMlr = []; CMsi = []; ED = []; v_min_maxSI = []; v_min_maxLR = []; v_min_maxAP = []; dsi = []
    dap = []; dlr = []; Ic = []; CMap2 = []; CMlr2 = []; CMsi2 = []; VOLUME = []; EDmm = []; VOL = []; MAL = []
    AREA = []; EXT = []; Ic_norm = []; STD = []; STD_norm = []; DIF_I = []; ENTROPY = []; Imax = []; DIF_I2 = []
    d_lr = 6; d_ap = 6; d_si = 3; Imitja = np.mean(LL[LL > 0]); Imaxim = np.amax(LL)
    VL = np.zeros((size_vol[0], size_vol[1], size_vol[2]))
    
    for i in range(len(CM)): 
        # Dur a terme el creixement en un entorn v_lr, v_ap, v_si de cada regió 
        v_lr = np.linspace(int(round(CM[i,2])) - d_lr, int(round(CM[i,2])) + d_lr, 2*d_lr + 1)
        v_ap = np.linspace(int(round(CM[i,1])) - d_ap, int(round(CM[i,1])) + d_ap, 2*d_ap + 1)
        v_si = np.linspace(int(round(CM[i,0])) - d_si, int(round(CM[i,0])) + d_si, 2*d_si + 1)
        L_little = np.zeros((2*d_si + 1, 2*d_ap + 1, 2*d_lr + 1))
        for jj in range(len(v_lr)):
            for j in range(len(v_ap)):
                for k in range(len(v_si)):
                    L_little[k,j,jj] = LL[int(v_si[k]),int(v_ap[j]),int(v_lr[jj])]

        FFLL = flood_fill(L_little, (d_si,d_ap,d_lr), 10000, tolerance=30) # Creixement amb flood_fill, píxels de la nova regió a 10000 
        f_si, f_ap, f_lr = np.where(FFLL == 10000)
        
        # Coordenades dels píxels de la nova regió a la matriu original
        f_si = f_si + int(CM[i,0]) - d_si
        f_ap = f_ap + int(CM[i,1]) - d_ap
        f_lr = f_lr + int(CM[i,2]) - d_lr

        n_pixels = len(f_lr)
        for j in range(len(f_lr)):
            VL[int(f_si[j])][int(f_ap[j])][int(f_lr[j])] = 1
        
    VL2,nregions = label(VL,return_num=True) # Etiquetatge de les noves regions
    props = regionprops(VL2, L)   
    
    for i in range(nregions):    
        n_pixels = props[i].area
        AREA.append(n_pixels) # Àrea
        ED.append(props[i].equivalent_diameter) # Diàmetre mig
        VOL.append(4/3*np.pi*(ED[-1]/2)**3) # Volum
        VOLUME.append(VOL[-1]*spacing[0]*spacing[1]*spacing[2]) # Volum en mm
        EDmm.append(2*(VOLUME[-1]/4*3/np.pi)**(1/3)) # Diàmetre mig en mm
        CM = props[i].weighted_centroid # Centroide ponderat
        CMap.append(CM[1]); CMlr.append(CM[2]); CMsi.append(CM[0])
        # Coordenades en mm respecte la carina
        CMap2.append((CM[1] - carina_p[0])*spacing[0]) 
        CMlr2.append((CM[2] - carina_p[1])*spacing[1])
        CMsi2.append((CM[0] - carina_p[2])*spacing[2])
        DIF_I2.append(props[i].max_intensity - props[i].min_intensity) # Diferència intensitat màxima - mínima
        cm = props[i].centroid # Centroide
        dsi.append(abs(CM[0] - cm[0])); dap.append(abs(CM[1] - cm[1])); dlr.append(abs(CM[2] - cm[2])) # Diferència centroides
        Imax.append(props[i].max_intensity) # Intensitat màxima
        Ic.append(props[i].mean_intensity) # Intensitat mitjana
        Ic_norm.append(props[i].mean_intensity/Imaxim) # Intensitat mitjana normalitzada
        DIF_I.append(props[i].mean_intensity - Imitja) # Diferència intensitat mitjana regió - intensitat mitjana TAC
        MAL.append(props[i].major_axis_length) # Longitud de l'eix major
        si0, ap0, lr0, si1, ap1, lr1 = props[i].bbox # Caixa de delimitació
        Lsi = si1 - si0 + 1; Lap = ap1 - ap0 + 1; Llr = lr1 - lr0 + 1 # Longitud dels costats de la caixa
        EXT.append(props[i].extent) # Extensió
        min_maxSI = Llr/Lap
        min_maxLR = Lap/Lsi
        min_maxAP = Lsi/Llr
        if min_maxSI > 1:
            v_min_maxSI.append(1/min_maxSI)
        else:
            v_min_maxSI.append(min_maxSI)
        if min_maxLR > 1:
            v_min_maxLR.append(1/min_maxLR)
        else:
            v_min_maxLR.append(min_maxLR)
        if min_maxAP > 1:
            v_min_maxAP.append(1/min_maxAP)
        else:
            v_min_maxAP.append(min_maxAP)

        # Mirar desviació típica i entropia en un entorn v_lr, v_ap, v_si
        v_lr = np.linspace(int(round(CM[2])) - d_lr, int(round(CM[2])) + d_lr, 2*d_lr + 1)
        v_ap = np.linspace(int(round(CM[1])) - d_ap, int(round(CM[1])) + d_ap, 2*d_ap + 1)
        v_si = np.linspace(int(round(CM[0])) - d_si, int(round(CM[0])) + d_si, 2*d_si + 1)

        v_intensity = []; v_intensity2 = []
        for jj in range(len(v_lr)):
            for j in range(len(v_ap)):
                for k in range(len(v_si)):
                    v_intensity.append(LL[int(v_si[k])][int(v_ap[j])][int(v_lr[jj])]) # Intensitat
                    v_intensity2.append(LL[int(v_si[k])][int(v_ap[j])][int(v_lr[jj])]/Imaxim) # Intensitat normalitzada

        v_intensity = np.array(v_intensity); v_intensity2 = np.array(v_intensity2)
        v_intensity = v_intensity[v_intensity > 0]; v_intensity2 = v_intensity2[v_intensity2 > 0]
        STD.append(v_intensity.std()); STD_norm.append(v_intensity2.std()) # Desviació típica

        H = 0; unique_elements, counts_elements = np.unique(v_intensity, return_counts=True)
        for j in range(len(unique_elements)):
            pj = counts_elements[j]/len(counts_elements)
            h = pj*np.log2(pj)
            H = H - h
        ENTROPY.append(H) # Entropia de Shannon
    
    
    c_tuples = list(zip(CMap,CMlr,CMsi,ED,v_min_maxSI,v_min_maxAP,v_min_maxLR,dsi,dap,dlr,Ic,CMap2,CMlr2,CMsi2,VOLUME,EDmm,VOL,AREA,EXT,STD,STD_norm,Ic_norm,DIF_I,ENTROPY,Imax,DIF_I2,MAL)) 
    C = sorted(c_tuples, key=lambda c: c[2]) # ordenar tot respecte CMz 

    CMap,CMlr,CMz,ED,v_min_maxSI,v_min_maxAP,v_min_maxLR,dsi,dap,dlr,Ic,CMap2,CMlr2,CMsi2,VOLUME,EDmm,VOL,AREA,EXT,STD,STD_norm,Ic_norm,DIF_I,ENTROPY,Imax,DIF_I2,MAL = zip(*C) # separar cada coordenada en un vector

    lrpl = CMlr; appl = CMap; sipl = CMsi # coordenades en píxels
    lrl = CMlr2; apl = CMap2; sil = CMsi2 # coordenades en mm
    
    if np.shape(lrc)[0] == 0: 
        print('Quantitat de FP: ' + str(len(lrl))) 
        # Crear llista TP o FP
        TPFP = []; nom = [];
        for i in range(len(lrl)):
            TPFP.append('NO')
            nom.append(NAMES[ii])

    else:
        vexp = []; vjj = []; vSS = []; vii = []
        for i in range(len(lrpl)):
            vj = []; vS = []
            for j in range(len(lrc)):
                S = np.sqrt((lrl[i] - lrc[j])**2 + (apl[i] - apc[j])**2 + (sil[i] - sic[j])**2)
                vS.append(S)
                vj.append(j)
            S_min = np.amin(vS); vSS.append(S_min)
            index = np.where(vS == S_min)[0][0]; vjj.append(vj[index])
            vii.append(i)

        dist_min = 4 # mm
        c_tuples = list(zip(vjj, vSS, vii)) 
        C = sorted(c_tuples, key=lambda c: c[0]) # ordenar tot respecte vjj 
        vjj, vSS, vii = zip(*C) # separar cada coordenada en un vector
        vSS = np.asarray(vSS)

        # True Positives: lesions detectades a una distància menor a dist_min d'una del radiòleg
        index1 = np.where(vSS <= dist_min)
        vl_TP = []; ve_TP = []; v_dist = []
        for i in index1[0]:
            vl_TP.append(vii[i])
            ve_TP.append(vjj[i])
            v_dist.append(vSS[i])

        vectori = [] # hi ha lesions del radioleg assignades a 2 meves, em quedo amb la més propera
        for i in range(len(ve_TP)-1):
            if ve_TP[i] == ve_TP[i+1]:
                S1 = v_dist[i]
                S2 = v_dist[i+1]
                if S1 < S2:
                    vectori.append(i)
                elif S1 > S2:
                    vectori.append(i+1)

        count = 0 
        for i in vectori:
            ve_TP.remove(ve_TP[i-count])
            vl_TP.remove(vl_TP[i-count])
            v_dist.remove(v_dist[i-count])
            count = count + 1 # cada cop que es fa remove s'acurta el vector
        print('Quantitat de TP: ' + str(len(vl_TP)))
        
        # False Positives: lesions detectades a una distància major a dist_min d'una del radiòleg
        vl_FP = np.setxor1d(np.arange(0,len(sil)), vl_TP) # diferència entre 2 conjunts, elements que hi ha a (vector amb nº lesions) però no a ve_TP
        print('Quantitat de FP: ' + str(np.size(vl_FP))) # print(ve_FN)

        # False Negatives: lesions detectades pel radiòleg però no per mi
        ve_FN = np.setxor1d(np.arange(0,len(sic)), ve_TP) # diferència entre 2 conjunts, elements que hi ha a (vector amb nº lesions) però no a ve_TP
        print('Quantitat de FN: ' + str(np.size(ve_FN))) # print(ve_FN)
        
        # Crear llista TP o FP
        YESNO = []; nom = [];
        for i in range(len(lrl)):
            if i in vl_TP:
                YESNO.append('YES')
            else:
                YESNO.append('NO')
            nom.append(NAMES[JJ])
            

    TOTAL_lrpl = TOTAL_lrpl + list(lrpl)
    TOTAL_appl = TOTAL_appl + list(appl)
    TOTAL_sipl = TOTAL_sipl + list(sipl)
    TOTAL_lrl = TOTAL_lrl + list(lrl)
    TOTAL_apl = TOTAL_apl + list(apl)
    TOTAL_sil = TOTAL_sil + list(sil)
    TOTAL_v_min_maxLR = TOTAL_v_min_maxLR + list(v_min_maxLR)
    TOTAL_v_min_maxAP = TOTAL_v_min_maxAP + list(v_min_maxAP)
    TOTAL_v_min_maxSI = TOTAL_v_min_maxSI + list(v_min_maxSI)
    TOTAL_dlr = TOTAL_dlr + list(dlr)
    TOTAL_dap = TOTAL_dap + list(dap)
    TOTAL_dsi = TOTAL_dsi + list(dsi)
    TOTAL_Ic = TOTAL_Ic + list(Ic)
    TOTAL_Imax = TOTAL_Imax + list(Imax)
    TOTAL_EDmm = TOTAL_EDmm + list(EDmm)
    TOTAL_AREA = TOTAL_AREA + list(AREA)
    TOTAL_EXT = TOTAL_EXT + list(EXT)
    TOTAL_STD = TOTAL_STD + list(STD)
    TOTAL_STD_norm = TOTAL_STD_norm + list(STD_norm)
    TOTAL_Ic_norm = TOTAL_Ic_norm + list(Ic_norm)
    TOTAL_DIF_I = TOTAL_DIF_I + list(DIF_I)
    TOTAL_DIF_I2 = TOTAL_DIF_I2 + list(DIF_I2)
    TOTAL_ENTROPY = TOTAL_ENTROPY + list(ENTROPY)
    TOTAL_MAL = TOTAL_MAL + list(MAL)
    TOTAL_YESNO = TOTAL_YESNO + YESNO
    TOTAL_nom = TOTAL_nom + nom

In [ ]:
# Crear arxiu Excel amb les diferents regions i els seus atributs

path = r"C:\Users\InmaV\Downloads\Lesions_Training_Pulmons_PLM_FF.xlsx"
df = pd.DataFrame({'L/R (mm)': TOTAL_lrl,
                   'A/P (mm)': TOTAL_apl,
                   'S/I (mm)': TOTAL_sil,
                   'min/max L/R': TOTAL_v_min_maxLR,
                   'min/max A/P': TOTAL_v_min_maxAP,
                   'min/max S/I': TOTAL_v_min_maxSI,
                   'dist L/R': TOTAL_dlr,
                   'dist A/P': TOTAL_dap,
                   'dist S/I': TOTAL_dsi,
                   'Area': TOTAL_AREA,
                   'Extent': TOTAL_EXT,
                   'STD': TOTAL_STD,
                   'STD norm': TOTAL_STD_norm,
                   'Int centre': TOTAL_Ic,
                   'Int centre norm': TOTAL_Ic_norm,
                   'Int maxima': TOTAL_Imax,
                   'Difference Int': TOTAL_DIF_I,
                   'Difference Int2': TOTAL_DIF_I2,
                   'Entropy': TOTAL_ENTROPY,
                   'DIAM (mm)': TOTAL_EDmm,
                   'YES/NO': TOTAL_YESNO,
                   'Pulmons': TOTAL_nom})
df = df[['L/R (mm)', 'A/P (mm)', 'S/I (mm)', 'min/max L/R', 'min/max A/P', 'min/max S/I', 'dist L/R', 'dist A/P', 'dist S/I', 'Area', 'Extent', 'STD', 'STD norm', 'Int centre', 'Int centre norm', 'Int maxima', 'Difference Int', 'Difference Int2', 'Entropy', 'DIAM (mm)', 'YES/NO', 'Pulmons']]
writer = pd.ExcelWriter(path, engine = 'xlsxwriter')

df.to_excel(writer, sheet_name = 'Pulmons')
writer.save()
writer.close()

In [ ]:
# Machine Learning: arbre de decisió

NAMES = [215379,218492,218493,218494,218508,218510,218529,218542,218552,218562,218563,315402,315404,315439]  

# Llegir l'Excel creat anteriorment
df = pd.read_excel('C:\\Users\\InmaV\\Downloads\\Lesions_Training_Pulmons_PLM_FF.xlsx',index_col=0)

# Codificació numèrica de les variables, transformar YES en 1 i NO en 0
encoder = LabelEncoder()
df['YES/NO'] = encoder.fit_transform(df['YES/NO'])
        
y = df['YES/NO'].values # Llista amb les etiquetes de cada regió
X = df.drop(['YES/NO','Pulmons'], axis=1).values # llista amb els atributs de cada regió

# Dividir les llistes en train/test aleatòriament, 70%/30%:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.30, random_state=21)

# Crear i entrenar arbre de decisió
clt = tree.DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=16, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')
clt = clt.fit(X_train,Y_train)

xx = df['Pulmons']; M_tn = []; M_tp = []; M_fn = []; M_fp = []
v_tn = []; v_tp = []; v_fn = []; v_fp = []
for i in range(len(NAMES)):
    index = np.where(xx == NAMES[i])[0]
    XX = X[index[0]:index[-1]+1]
    Y_test = y[index[0]:index[-1]+1]
    Y_pred = clt.predict(XX)
    cm = confusion_matrix(Y_test, Y_pred)
    tn, fp, fn, tp = cm.ravel()
    v_tn.append(tn); v_tp.append(tp); v_fn.append(fn); v_fp.append(fp)
M_tn.append(sum(v_tn)); M_tp.append(sum(v_tp)); M_fn.append(sum(v_fn)); M_fp.append(sum(v_fp))
TP = np.asarray(M_tp); FP = np.asarray(M_fp); FN = np.asarray(M_fn); TN = np.asarray(M_tn)
sensitivity = TP/(TP+FN)
specificity = TN/(TN+FP)
precision = TP/(TP+FP)
NPV = TN/(TN+FN) # NEGATIVE PREDICTIVE VALUE
accuracy = (TP+TN)/(TP+TN+FP+FN)
print(TP,FP,FN,TN,sensitivity,specificity,precision,NPV,accuracy)